# Feature Engineering

In [162]:
import pandas as pd
import numpy as np

In [163]:
df = pd.read_csv(r"AFL_Preprocessed.csv")
df = df.drop(columns="Unnamed: 0")

## Create margin feature

In [164]:
df["Margin"] = 0
score = 0
margins = []
crowd = 0
for index, row in df.iterrows():
    if index%2 == 0:
        score = row['FinalScore']
        margins.append(0)
        crowd = row['ActualCrowd']
    elif crowd == row['ActualCrowd']:
        margins.append(row['FinalScore'] - score)
    else:
        margins.append(1000)
df['Margin'] = margins
df[df['Margin'] == 1000]

,Team,Round,HomeAway,FinalScore,Date,Day,Time,Venue,ActualCrowd,Season,Opposition,WinLoss,LadderPosition,Margin
411,Essendon,1,Away,40,2019-03-24,Sunday,3,Sydney Showground,15434,2019,Greater Western Sydney,Loss,18,1000
533,Adelaide,8,Away,88,2019-05-11,Saturday,7,Adelaide Oval,49765,2019,Port Adelaide,Win,3,1000
641,Western Bulldogs,15,Away,66,2019-06-29,Saturday,7,Adelaide Oval,25362,2019,Port Adelaide,Win,13,1000
691,Collingwood,18,Away,75,2019-07-20,Saturday,5,Sydney Showground,15476,2019,Greater Western Sydney,Loss,4,1000


These instances with the crowd at each game have different amounts  only differ by a few people, thus we are going to say they are approimately the same and treat them as such

In [165]:
df["Margin"] = 0
score = 0
margins = []
crowd = 0
for index, row in df.iterrows():
    if index%2 == 0:
        score = row['FinalScore']
        margins.append(0)
        crowd = row['ActualCrowd']
    else:
        margins.append(row['FinalScore'] - score)

df['Margin'] = margins

df['margin_shifted'] = -df['Margin'].shift(-1)
df['margin_shifted'] = df['margin_shifted'].fillna(0)
df['Margin'] = df['Margin'] + df['margin_shifted']
df = df.drop(columns="margin_shifted")
df['Margin'] = df['Margin'].astype(int)
df

,Team,Round,HomeAway,FinalScore,Date,Day,Time,Venue,ActualCrowd,Season,Opposition,WinLoss,LadderPosition,Margin
0,Melbourne,1,Home,97,2022-03-16,Wednesday,7,MCG,58002,2022,Western Bulldogs,Win,2,26
1,Western Bulldogs,1,Away,71,2022-03-16,Wednesday,7,MCG,58002,2022,Melbourne,Loss,17,-26
2,Carlton,1,Home,101,2022-03-17,Thursday,7,MCG,72179,2022,Richmond,Win,5,25
3,Richmond,1,Away,76,2022-03-17,Thursday,7,MCG,72179,2022,Carlton,Loss,14,-25
4,St Kilda,1,Home,85,2022-03-18,Friday,8,Docklands,40129,2022,Collingwood,Loss,12,-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345,St Kilda,24,Away,86,2011-09-03,Saturday,7,MCG,55606,2011,Carlton,Loss,6,20
4346,Port Adelaide,24,Home,112,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Melbourne,Win,16,8
4347,Melbourne,24,Away,104,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Port Adelaide,Win,13,-8
4348,Richmond,24,Home,91,2011-09-04,Sunday,5,Docklands,32890,2011,North Melbourne,Loss,12,-13


From this feature we can actually see that some of the wins ascoiated with team is incorrect as they have a megative margin (meaning that lossed the match)

We can now clean this WinLoss column up (This is fine to do as when we cleaned the data before anytime we removed a match we removed it counterpart)

In [166]:
print(df.WinLoss.value_counts())
df.loc[np.sign(df["Margin"]) == 1, "WinLoss"] = 'Win'
df.loc[np.sign(df["Margin"]) == -1, "WinLoss"] = 'Loss'
print(df.WinLoss.value_counts())
df

Win     2187
Loss    2163
Name: WinLoss, dtype: int64
Loss    2182
Win     2168
Name: WinLoss, dtype: int64


,Team,Round,HomeAway,FinalScore,Date,Day,Time,Venue,ActualCrowd,Season,Opposition,WinLoss,LadderPosition,Margin
0,Melbourne,1,Home,97,2022-03-16,Wednesday,7,MCG,58002,2022,Western Bulldogs,Win,2,26
1,Western Bulldogs,1,Away,71,2022-03-16,Wednesday,7,MCG,58002,2022,Melbourne,Loss,17,-26
2,Carlton,1,Home,101,2022-03-17,Thursday,7,MCG,72179,2022,Richmond,Win,5,25
3,Richmond,1,Away,76,2022-03-17,Thursday,7,MCG,72179,2022,Carlton,Loss,14,-25
4,St Kilda,1,Home,85,2022-03-18,Friday,8,Docklands,40129,2022,Collingwood,Loss,12,-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345,St Kilda,24,Away,86,2011-09-03,Saturday,7,MCG,55606,2011,Carlton,Win,6,20
4346,Port Adelaide,24,Home,112,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Melbourne,Win,16,8
4347,Melbourne,24,Away,104,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Port Adelaide,Loss,13,-8
4348,Richmond,24,Home,91,2011-09-04,Sunday,5,Docklands,32890,2011,North Melbourne,Loss,12,-13


The 14 extra losses in the dataframe correlate with 7 extra matches which can be accounted for a draw counting as a loss for both teams.

## Opponents Ladder Position

In [167]:
df["OppLadderPosition"] = 0
score = 0
LadderPos = []
for index, row in df.iterrows():
    if index%2 == 0:
        LadPos = row['LadderPosition']
        LadderPos.append(0)
    else:
        LadderPos.append(LadPos)
df['OppLadderPosition'] = LadderPos

df['ladder_shifted'] = df['LadderPosition'].shift(-1)
df['ladder_shifted'] = df['ladder_shifted'].fillna(0)

ladder = []
for index, row in df.iterrows():
    if row['OppLadderPosition'] == 0:
        ladder.append(row['ladder_shifted'])
    else:
        ladder.append(0)
df['ladder_shifted'] = ladder
df['OppLadderPosition'] = df['OppLadderPosition'] + df['ladder_shifted']
df['OppLadderPosition'] = df['OppLadderPosition'].astype(int)
df = df.drop(columns="ladder_shifted")
df

,Team,Round,HomeAway,FinalScore,Date,Day,Time,Venue,ActualCrowd,Season,Opposition,WinLoss,LadderPosition,Margin,OppLadderPosition
0,Melbourne,1,Home,97,2022-03-16,Wednesday,7,MCG,58002,2022,Western Bulldogs,Win,2,26,17
1,Western Bulldogs,1,Away,71,2022-03-16,Wednesday,7,MCG,58002,2022,Melbourne,Loss,17,-26,2
2,Carlton,1,Home,101,2022-03-17,Thursday,7,MCG,72179,2022,Richmond,Win,5,25,14
3,Richmond,1,Away,76,2022-03-17,Thursday,7,MCG,72179,2022,Carlton,Loss,14,-25,5
4,St Kilda,1,Home,85,2022-03-18,Friday,8,Docklands,40129,2022,Collingwood,Loss,12,-17,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345,St Kilda,24,Away,86,2011-09-03,Saturday,7,MCG,55606,2011,Carlton,Win,6,20,5
4346,Port Adelaide,24,Home,112,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Melbourne,Win,16,8,13
4347,Melbourne,24,Away,104,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Port Adelaide,Loss,13,-8,16
4348,Richmond,24,Home,91,2011-09-04,Sunday,5,Docklands,32890,2011,North Melbourne,Loss,12,-13,9


## Last 5 Games feature

In [168]:
df = df.sort_values(by=['Team', 'Season', 'Round'])
df['Last5'] = 0

for team in df['Team'].unique():
    team_mask = df['Team'] == team
    team_wins = df.loc[team_mask, 'WinLoss'].eq('Win').rolling(6, min_periods=1).sum() - df.loc[team_mask, 'WinLoss'].eq('Win')
    df.loc[team_mask, 'Last5'] = team_wins
df['Last5'] = df['Last5'].astype(int)
df = df.sort_index()
df

,Team,Round,HomeAway,FinalScore,Date,Day,Time,Venue,ActualCrowd,Season,Opposition,WinLoss,LadderPosition,Margin,OppLadderPosition,Last5
0,Melbourne,1,Home,97,2022-03-16,Wednesday,7,MCG,58002,2022,Western Bulldogs,Win,2,26,17,0
1,Western Bulldogs,1,Away,71,2022-03-16,Wednesday,7,MCG,58002,2022,Melbourne,Loss,17,-26,2,4
2,Carlton,1,Home,101,2022-03-17,Thursday,7,MCG,72179,2022,Richmond,Win,5,25,14,2
3,Richmond,1,Away,76,2022-03-17,Thursday,7,MCG,72179,2022,Carlton,Loss,14,-25,5,5
4,St Kilda,1,Home,85,2022-03-18,Friday,8,Docklands,40129,2022,Collingwood,Loss,12,-17,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345,St Kilda,24,Away,86,2011-09-03,Saturday,7,MCG,55606,2011,Carlton,Win,6,20,5,3
4346,Port Adelaide,24,Home,112,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Melbourne,Win,16,8,13,0
4347,Melbourne,24,Away,104,2011-09-04,Sunday,3,Adelaide Oval,29340,2011,Port Adelaide,Loss,13,-8,16,1
4348,Richmond,24,Home,91,2011-09-04,Sunday,5,Docklands,32890,2011,North Melbourne,Loss,12,-13,9,3


In [169]:
df.to_csv("AFL.csv")